## Regressão Linear Simples

In [ ]:
import pandas as pd
import numpy as np
import statsmodels.api as sm
import scipy.stats as stats

import matplotlib.pyplot as plt
import seaborn as sns

import graphs

In [ ]:
plt.rcParams['figure.figsize'] = [12, 8]

Vamos ler o dataset e ver algumas estatísticas

In [ ]:
cars = pd.read_csv('cars.csv')
cars.head()

In [ ]:
cars.describe().T

In [ ]:
cars.corr()

Vamos plotar alguns gráficos

In [ ]:
plt.hist(cars['speed'], bins=5)
plt.xlabel('Velocidade em MPH')
plt.ylabel('Fequência')
plt.title('Histograma da Velocidade');

In [ ]:
plt.hist(cars['dist'], bins=5)
plt.xlabel('Distância em Pés')
plt.ylabel('Fequência')
plt.title('Histograma da Distância');

In [ ]:
plt.scatter(cars['speed'], cars['dist'])
plt.xlabel('Velocidade')
plt.ylabel('Distância')
plt.title('Gráfico de Dispersão');

In [ ]:
beta_1 = sum((cars['speed'] - cars['speed'].mean()) * (cars['dist'] - cars['dist'].mean())) /\
        sum((cars['speed'] - cars['speed'].mean())**2)
beta_1

In [ ]:
beta_0 = cars['dist'].mean() - beta_1 * cars['speed'].mean()
beta_0

In [ ]:
plt.scatter(cars['speed'], cars['dist'])
plt.xlabel('Velocidade')
plt.ylabel('Distância')
plt.title('Gráfico de Dispersão');
plt.plot(cars['speed'], beta_0 + beta_1 * cars['speed'], color='red')

In [ ]:
residuals = cars['dist'] - (beta_0 + beta_1 * cars['speed'])

In [ ]:
sum(residuals)

In [ ]:
X = sm.add_constant(cars['speed'])
y = cars['dist']
X.head()

In [ ]:
model = sm.OLS(y, X).fit()

In [ ]:
model.summary()

#### Equação

A equação pode ser construída pela saída do modelo:

$$Distancia = -17.6 + (3.9)*Velocidade$$

#### Interpretação dos coeficientes

Coeficiente de inclinação: para um aumento de uma milha por hora na velocidade do carro, a distância, na média, que percorre para parar aumenta de aproximadamente 3.9

Coeficiente de interceptação: quando um carro está a 0 MPH, a distância, na média, que um carro percorre para parar é de aproximadamente -17.6 pés. Faz sentido esse valor?

#### $R^2$

O coeficiente de determinação é 0.65, ou seja, 65% da variabilidade da distância é explicado pela variável velocidade

#### Erro residual padrão

In [ ]:
np.sqrt(model.scale)

In [ ]:
np.sqrt(sum((cars['dist'] - (beta_0 + beta_1 * cars['speed']))**2) / 48)

O erro residual padrão é 15.38. Esse valor é uma aproximação de quanto os resíduos tendem a desviar da linha da regressão

#### Significância

Podemos ver que todos os coeficientes são significativos. A estatística F para a regressão também é significativa.

#### Intervalo de confiança

Podemos ver também o intervalo de confiança de 95% dos coeficientes

#### Verificando as premissas

##### Linearidade

In [ ]:
plt.scatter(cars['speed'], cars['dist'])
plt.xlabel('Velocidade')
plt.ylabel('Distância')
plt.title('Gráfico de Dispersão');
plt.plot(cars['speed'], beta_0 + beta_1 * cars['speed'], color='red');

##### Homocedasticidade e independência dos erros

In [ ]:
plt.scatter(model.fittedvalues, model.resid)
plt.title('Gráfico dos Resíduos')
plt.xlabel('Valores Ajustados')
plt.ylabel('Valores dos Resíduos')
plt.axhline(color='red');

##### Normalidade

In [ ]:
graph_plotter = graphs.AssumptionGraphs(model)

In [ ]:
graph_plotter.plot_qq()

#### Outros gráficos

##### Gráfico dos resíduos

Gráfico de dispersão dos valores ajustados contra os resíduos com uma linha suavizada de regressão mostrando se há alguma tendência.

In [ ]:
graph_plotter.plot_residual_fitted_values(y)

Outliers são observações em que temos grandes valores residuais. O erro para essas observações é relativamente grande porque a observação está distante da linha de regressão

##### Gráfico Scale-Location

Outro gráfico dos resíduos mostrando sua dispersão. Pode ser usado para avaliar a heterocedasticidade

In [ ]:
graph_plotter.plot_scale_location()

Queremos avaliar se a linha vermelha é aproximadamente horizontal, ou seja, que a média da magnitude dos resíduos padronizados não está mudando muito em função dos valores ajustados. E queremos avaliar que a dispersão em volta da linha vermelha não varia muito com os valores ajustados.

##### Gráfico de Influência

Esse gráfico mostra se algum outlier tem influência no ajuste da regressão. Tudo fora do grupo ou fora da linha da distância de Cook pode ter influência no ajuste do modelo

In [ ]:
graph_plotter.plot_influence()

Pontos influentes são observações que tem valores de uma variável independente pequenos ou grandes, ou seja esses pontos caem longe da média. Portanto, essas observações tem uma influência grande para mudar a inclinação da linha de regressão. Quanto mais longe uma observação tiver da média da variável independente, mais influência terá na inclinação.

A distância de Cook ajudar a medir o efeito de remover uma observação do dataset e retreinar o modelo. Observações que tem um valor residual alto e também uma alta influência tendem a ameaçar a acurária da linha de regressão e assim precisam ser melhor investigados.

##### Outro gráfico de influência

In [ ]:
model.get_influence().plot_influence();

#### Intervalos de confiança

In [ ]:
confidence_interval = model.get_prediction(X).conf_int()
prediction_interval = model.get_prediction(X).conf_int(obs=True)

In [ ]:
plt.scatter(cars['speed'], cars['dist'])
plt.xlabel('Velocidade')
plt.ylabel('Distância')
plt.title('Gráfico de Dispersão');
plt.plot(cars['speed'], beta_0 + beta_1 * cars['speed'], color='red', label='Regressão')
plt.plot(cars['speed'], confidence_interval[:,0], ls=':', color='green', label='Intervalo de Confiança')
plt.plot(cars['speed'], confidence_interval[:,1], ls=':', color='green')
plt.plot(cars['speed'], prediction_interval[:,0], ls=':', color='black', label='Intervalo de Previsão')
plt.plot(cars['speed'], prediction_interval[:,1], ls=':', color='black')
plt.legend();

#### Transformação Box-Cox

In [ ]:
y_transformed, fitted_lambda = stats.boxcox(y)

In [ ]:
fitted_lambda

In [ ]:
model2 = sm.OLS(y_transformed, X).fit()

In [ ]:
model2.summary()

In [ ]:
stats.probplot(model2.resid, dist="norm", plot=plt);

In [ ]:
graph_plotter = graphs.AssumptionGraphs(model2)

In [ ]:
graph_plotter.plot_residual_fitted_values(y_transformed)

In [ ]:
graph_plotter.plot_scale_location()

In [ ]:
graph_plotter.plot_influence()